# 10_Vectores_Academicos
Este cuaderno genera un CSV con 23 carreras y 69 columnas (habilidades técnicas agrupadas por HDBSCAN) a partir de los CSV de ofertas por carrera. Para 'CIENCIA DE DATOS E INTELIGENCIA ARTIFICIAL' se unen los datos de Ciencia_de_Datos e Inteligencia_Artificial.

In [1]:
# Importaciones
import os
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
# Cargar datos procesados comunes (habilidades whitelist, grupos HDBSCAN, nombres de dimensiones)
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)
habilidades_whitelist = datos.get('habilidades')
grupos_bge_ngram = datos.get('grupos_bge_ngram')
tfidf_epn_69d = datos.get('tfidf_epn_69d')
# Nombres legibles de las 69 dimensiones (si existen en tfidf_epn_69d)
if tfidf_epn_69d is not None:
    dim_names = list(tfidf_epn_69d.T.columns)
else:
    # Fallback: usar etiquetas de grupos HDBSCAN como nombres
    dim_names = [str(k) for k in grupos_bge_ngram.keys()]
print(f'Dimensiones técnicas: {len(dim_names)}')

Dimensiones técnicas: 69


In [3]:
# Función: vectorizar una carrera desde uno o más CSVs (skills + description → 69D técnico)
def vectorizar_carrera_desde_csvs(rutas_csv, habilidades_whitelist, grupos_bge_ngram):
    dfs = []
    for ruta in rutas_csv:
        if not os.path.exists(ruta):
            print(f'Archivo no encontrado: {ruta}')
            continue
        df = pd.read_csv(ruta, low_memory=False)
        dfs.append(df)
    if not dfs:
        return np.zeros(len(grupos_bge_ngram))
    df_all = pd.concat(dfs, ignore_index=True)
    # Asegurar columnas necesarias
    for col in ['skills', 'description']:
        if col not in df_all.columns:
            df_all[col] = ''
    textos = df_all[['skills','description']].fillna('').agg(' '.join, axis=1).str.lower().tolist()
    # Matriz término-documento en vocabulario de habilidades técnicas
    vectorizer = CountVectorizer(vocabulary=habilidades_whitelist, analyzer='word', ngram_range=(1, 5), lowercase=True)
    X = vectorizer.fit_transform(textos)
    matriz_td = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out()).T
    # Agregar términos por cada grupo HDBSCAN para construir 69 dimensiones
    matriz_69d = pd.DataFrame(0, index=list(grupos_bge_ngram.keys()), columns=range(len(textos)))
    for label, terms in grupos_bge_ngram.items():
        terms_validos = [t for t in terms if t in matriz_td.index]
        if terms_validos:
            matriz_69d.loc[label] = matriz_td.loc[terms_validos].sum(axis=0)
    # TF-IDF sobre las 69 dimensiones por documento
    tfidf = TfidfTransformer()
    tfidf_69d = tfidf.fit_transform(matriz_69d.values)
    tfidf_69d_df = pd.DataFrame(tfidf_69d.toarray(), index=matriz_69d.index, columns=matriz_69d.columns).T
    # Vector representativo de la carrera: promedio sobre documentos
    vector_carrera_69d = tfidf_69d_df.mean(axis=0).values
    return vector_carrera_69d

In [4]:
# Definición de las 23 carreras y mapeo a carpetas/archivos
BASE = 'todas_las_plataformas'
carreras_23 = [
    'ADMINISTRACIÓN DE EMPRESAS',
    'AGROINDUSTRIA',
    'CIENCIA DE DATOS E INTELIGENCIA ARTIFICIAL',
    'COMPUTACIÓN',
    'ECONOMÍA',
    'ELECTRICIDAD',
    'ELECTRÓNICA Y AUTOMATIZACIÓN',
    'FÍSICA',
    'GEOLOGÍA',
    'INGENIERÍA AMBIENTAL',
    'INGENIERÍA CIVIL',
    'INGENIERÍA DE LA PRODUCCIÓN',
    'INGENIERÍA QUÍMICA',
    'MATEMÁTICA',
    'MATEMÁTICA APLICADA',
    'MATERIALES',
    'MECÁNICA',
    'MECATRÓNICA',
    'PETRÓLEOS',
    'SISTEMAS DE INFORMACIÓN',
    'SOFTWARE',
    'TECNOLOGÍAS DE LA INFORMACIÓN',
    'TELECOMUNICACIONES'
]
folder_map = {
    'ADMINISTRACIÓN DE EMPRESAS': 'Administración_de_Empresas',
    'AGROINDUSTRIA': 'Agroindustria',
    'COMPUTACIÓN': 'Computación',
    'ECONOMÍA': 'Economía',
    'ELECTRICIDAD': 'Electricidad',
    'ELECTRÓNICA Y AUTOMATIZACIÓN': 'Electrónica_y_Automatización',
    'FÍSICA': 'Física',
    'GEOLOGÍA': 'Geología',
    'INGENIERÍA AMBIENTAL': 'Ingeniería_Ambiental',
    'INGENIERÍA CIVIL': 'Ingeniería_Civil',
    'INGENIERÍA DE LA PRODUCCIÓN': 'Ingeniería_de_la_Producción',
    'INGENIERÍA QUÍMICA': 'Ingeniería_Química',
    'MATEMÁTICA': 'Matemática',
    'MATEMÁTICA APLICADA': 'Matemática_Aplicada',
    'MATERIALES': 'Materiales',
    'MECÁNICA': 'Mecánica',
    'MECATRÓNICA': 'Mecatrónica',
    'PETRÓLEOS': 'Petróleos',
    'SISTEMAS DE INFORMACIÓN': 'Sistemas_de_Información',
    'SOFTWARE': 'Software',
    'TECNOLOGÍAS DE LA INFORMACIÓN': 'Tecnologías_de_la_Información',
    'TELECOMUNICACIONES': 'Telecomunicaciones'
}
def ruta_merged(folder):
    return os.path.join(BASE, folder, f'{folder}_Merged.csv')

In [5]:
# Calcular vectores 69D por carrera
resultados = {}
for carrera in carreras_23:
    if carrera == 'CIENCIA DE DATOS E INTELIGENCIA ARTIFICIAL':
        rutas = [
            os.path.join(BASE, 'Ciencia_de_Datos', 'Ciencia_de_Datos_Merged.csv'),
            os.path.join(BASE, 'Inteligencia_Artificial', 'Inteligencia_Artificial_Merged.csv'),
        ]
    else:
        folder = folder_map[carrera]
        rutas = [ruta_merged(folder)]
    vec = vectorizar_carrera_desde_csvs(rutas, habilidades_whitelist, grupos_bge_ngram)
    resultados[carrera] = vec
# Construir DataFrame con nombres legibles de dimensiones
df_vectores = pd.DataFrame.from_dict(resultados, orient='index')
df_vectores.columns = dim_names
df_vectores.index.name = 'CARRERA'
print('Shape final:', df_vectores.shape)
df_vectores.head()

Shape final: (23, 69)


,"administración de empresas, gestión de calidad, gestión de datos ...","agroindustria, agronomía","análisis de datos, análisis de materiales, análisis de sistemas ...","aprendizaje no supervisado, aprendizaje supervisado","arcgis, qgis",arquitectura de computadoras,"aws, wireless",balances de materia y energía,base de datos,buenas prácticas agrícolas,...,"teoría cuántica, teoría de juegos, teoría de números","termodinámica, termodinámica química","tratamiento de aguas, tratamientos térmicos",visualización de datos,planificación de la producción,planificación de obras,procesamiento de alimentos,procesamiento de lenguaje natural,redes,redes neuronales
CARRERA,,,,,,,,,,,,,,,,,,,,,
ADMINISTRACIÓN DE EMPRESAS,0.001982,0.000192,0.000666,0.00000,0.000192,0.000000,0.000740,0.0,0.000845,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000819,0.000000
AGROINDUSTRIA,0.002099,0.002643,0.001021,0.00000,0.000000,0.000000,0.000000,0.0,0.000592,0.001025,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000592,0.000000,0.001565,0.000000
CIENCIA DE DATOS E INTELIGENCIA ARTIFICIAL,0.000745,0.000113,0.001399,0.00018,0.000149,0.000000,0.000885,0.0,0.000510,0.000000,...,0.000086,0.0,0.0,0.000677,0.000121,0.0,0.000000,0.000324,0.000612,0.000397
COMPUTACIÓN,0.001253,0.000000,0.001728,0.00000,0.000000,0.000317,0.001975,0.0,0.000943,0.000000,...,0.000000,0.0,0.0,0.000486,0.000000,0.0,0.000000,0.000000,0.001100,0.000000
ECONOMÍA,0.001268,0.000564,0.001192,0.00000,0.000399,0.000000,0.000797,0.0,0.000399,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000564,0.000000


In [7]:
# Guardar CSV en el directorio raíz del proyecto
salida_csv = 'Vectores_Academicos_69d.csv'
df_vectores.to_csv(salida_csv, index=True, encoding='utf-8')
print(f'CSV guardado: {salida_csv}')

CSV guardado: Vectores_Academicos_69d.csv
